# QiskitRuntimeService - Complete Guide for Qiskit Certification

**Section 4: Run Circuits - Part 2**

> **Exam Weight**: Part of 15% | **Must Master**: ✅✅

This notebook covers QiskitRuntimeService - your gateway to IBM Quantum hardware and cloud services!

---

## 🎯 Learning Objectives

By the end of this notebook, you will:
- Connect to IBM Quantum using QiskitRuntimeService
- Save and manage API credentials
- List and select backends
- Use `least_busy()` for optimal backend selection
- Understand the IBM Quantum architecture

---

## 💡 Conceptual Deep Dive: The Airline Check-In Analogy

**QiskitRuntimeService = Airport Check-In System**

| Airport | IBM Quantum |
|---------|-------------|
| **Your Ticket (Token)** | IBM Quantum API token |
| **Check-In Desk** | `QiskitRuntimeService()` |
| **Flight Options** | `service.backends()` |
| **Seat Selection** | Choose backend by qubits/queue |
| **Boarding** | `Session` reserves hardware |

```
Your Code (Local)
    ↓
┌──────────────────────┐
│  QiskitRuntimeService│  ← Authentication
└──────────────────────┘
    ↓ (Internet)
┌──────────────────────┐
│  IBM Quantum Cloud   │  ← Queue, scheduling
└──────────────────────┘
    ↓
┌──────────────────────┐
│  Quantum Backend     │  ← Real QPU or Simulator
└──────────────────────┘
```

---

## Setup

Import required libraries:

In [ ]:
# Note: This notebook demonstrates patterns - you need your own IBM Quantum token
# Get token from: https://quantum.ibm.com/account

from qiskit_ibm_runtime import QiskitRuntimeService

print("✅ Import successful!")
print("\n⚠️ To run examples, you need an IBM Quantum account and token")

## Part 1: Three Initialization Patterns

### Three Initialization Patterns

In [ ]:
# PATTERN 1: Save account (one-time setup)
QiskitRuntimeService.save_account(
    channel='ibm_quantum_platform',
    token='',
    overwrite=True
)

print("Pattern 1: Save account for future use")
print("  - Run once to store credentials")
print("  - channel='ibm_quantum_platform' for free tier")
print("  - overwrite=True to update existing")

In [ ]:
# PATTERN 2: Default initialization (uses saved credentials)
# Make sure cell 4 was executed first to save credentials
service = QiskitRuntimeService()

print("Pattern 2: Load saved credentials")
print("  - Uses credentials saved with save_account()")
print("  - Most common pattern after initial setup")
print("  - No need to expose token in code")

In [ ]:
# PATTERN 3: Explicit token (not recommended for production)
print("Pattern 3: Explicit token")
print("""
service = QiskitRuntimeService(
    channel='ibm_quantum_platform',
    token='YOUR_IBM_QUANTUM_TOKEN_HERE'
)

⚠️ Note: This pattern embeds the token in code
✓ Use for: Testing, one-off scripts
✗ Avoid for: Production, shared code
✗ Never: Commit tokens to git!
""")

### ⚠️ EXAM TRAP: Channels

```python
# Two channel options:
channel='ibm_quantum_platform'  # ✓ Free IBM Quantum Platform (most common)
channel='ibm_cloud'             # ✓ IBM Cloud (paid, enterprise)

# Common mistakes:
channel='ibm_quantum'  # ❌ WRONG (old name, deprecated)
channel='quantum'      # ❌ WRONG
channel='ibm'          # ❌ WRONG
```

**EXAM TIP**: Use `'ibm_quantum_platform'` for certification!

### 💡 Local Simulation Alternative (No Account Needed)

For most learning and development, you can use **local simulators** without an IBM account:

```python
from qiskit_aer import AerSimulator

# No internet, no token needed!
backend = AerSimulator()

# Works for: Transpilation, basic execution, Options testing
# Does NOT work for: IBM Runtime features (Sessions, Batch, resilience_level)
```

## Part 2: Backend Selection

In [ ]:
# Assuming service is initialized
# service = QiskitRuntimeService()

# List all available backends
print("Method 1: List all backends")
print("  backends = service.backends()")
print("  Returns: List of backend objects")
print()

# Example output:
print("Example backends:")
print("  - ibm_brisbane (127 qubits)")
print("  - ibm_kyoto (127 qubits)")
print("  - ibmq_qasm_simulator (simulator)")

In [ ]:
# Get specific backend by name
print("Method 2: Get specific backend")
print("  backend = service.backend('ibm_brisbane')")
print("  Returns: Single backend object")
print()

# Common backend names (exam may reference these):
print("Common backend names:")
print("  - 'ibm_brisbane' - 127-qubit Heron processor")
print("  - 'ibm_kyoto' - 127-qubit Eagle processor")
print("  - 'ibmq_qasm_simulator' - Cloud simulator")

In [ ]:
# Find least busy backend
print("Method 3: Auto-select least busy")
print("  backend = service.least_busy(")
print("      operational=True,")
print("      simulator=False")
print("  )")
print()

print("Parameters:")
print("  operational=True  → Only working backends")
print("  simulator=False   → Exclude simulators (real hardware only)")
print("  simulator=True    → Only simulators")
print()
print("🎯 EXAM TIP: least_busy() finds available backend with shortest queue!")

## Part 3: Backend Properties (EXAM CRITICAL!)

In [ ]:
# Backend properties pattern
# backend = service.backend('ibm_brisbane')

print("Essential backend properties:")
print()

print("1. backend.name")
print("   → String: 'ibm_brisbane'")
print()

print("2. backend.num_qubits")
print("   → Integer: 127")
print()

print("3. backend.operation_names  (V2 API)")
print("   → List: ['ecr', 'id', 'rz', 'sx', 'x', 'measure', 'delay']")
print("   → Replaces backend.configuration().basis_gates (V1)")
print()

print("4. backend.coupling_map")
print("   → List of tuples: [(0,1), (1,0), (1,2), ...]")
print("   → Shows which qubits can interact directly")
print()

print("5. backend.target")
print("   → Target object with detailed operation specs")
print("   → V2 API comprehensive info")

### ⚠️ EXAM TRAP: V1 vs V2 API

```python
# V1 (DEPRECATED)
backend.configuration().basis_gates  # ❌ Old API
backend.properties()                 # ❌ Old API

# V2 (CURRENT - Use This!)
backend.operation_names              # ✓ New API
backend.target                       # ✓ New API
```

**EXAM TIP**: Qiskit 1.0+ uses V2 backend API!

## Part 4: Simulator vs Hardware

In [ ]:
# Identifying simulators
print("How to identify simulators:")
print()

print("1. Name contains 'simulator'")
print("   - 'ibmq_qasm_simulator' → Simulator ✓")
print("   - 'simulator_statevector' → Simulator ✓")
print("   - 'ibm_brisbane' → Real hardware")
print()

print("2. Use simulator parameter in least_busy()")
print("   service.least_busy(simulator=False) → Real hardware only")
print("   service.least_busy(simulator=True) → Simulators only")
print()

print("3. Check backend properties")
print("   Real hardware: Limited qubits, connectivity constraints")
print("   Simulators: May have no connectivity constraints")

### Simulator vs Hardware Comparison

| Feature | Simulator | Real Hardware |
|---------|-----------|---------------|
| Noise | None (ideal) | Yes (decoherence, gates) |
| Queue | Short/instant | Can be hours |
| Cost | Free | Uses credits |
| Accuracy | Perfect | Error-prone |
| Connectivity | Often all-to-all | Limited (coupling map) |
| Use case | Testing, learning | Real experiments |

**EXAM TIP**: Simulators perfect for testing, hardware for real results!

### 🎓 Exam Question Patterns - Backend Selection

**Pattern 1: "Which backend for...?"**
```
Fastest testing → 'ibmq_qasm_simulator'
Real quantum effects → least_busy(simulator=False)
Specific hardware → service.backend('ibm_brisbane')
Noise-free ideal → StatevectorSimulator (local)
```

**Pattern 2: "How to find available backends?"**
```python
# All backends
backends = service.backends()

# With filters
backends = service.backends(
    filters=lambda x: x.num_qubits >= 5 and not x.simulator
)

# Least busy
backend = service.least_busy(operational=True, simulator=False)
```

## Part 5: Common Patterns

In [ ]:
# Pattern 1: Quick simulator access
print("Pattern 1: Get simulator")
print("""
service = QiskitRuntimeService()
backend = service.backend('ibmq_qasm_simulator')
""")
print("✓ Fast, free, ideal for testing")
print()

In [ ]:
# Pattern 2: Auto-select best hardware
print("Pattern 2: Find best available hardware")
print("""
service = QiskitRuntimeService()
backend = service.least_busy(
    operational=True,
    simulator=False,
    min_num_qubits=5
)
""")
print("✓ Automatically finds shortest queue")
print()

In [ ]:
# Pattern 3: Specific hardware selection
print("Pattern 3: Choose specific hardware")
print("""
service = QiskitRuntimeService()
backend = service.backend('ibm_brisbane')

# Verify it's what you want
print(f"Backend: {backend.name}")
print(f"Qubits: {backend.num_qubits}")
""")
print("✓ Use when you need specific processor")

## 📝 Practice Questions

### Question 1: Service Initialization

**What does `QiskitRuntimeService()` with no arguments do?**

A) Throws an error - token required  
B) Uses saved credentials from save_account()  
C) Connects to free simulator only  
D) Prompts for token interactively

<details>
<summary>Answer</summary>

**B) Uses saved credentials from save_account()**

```python
# First time: save credentials
QiskitRuntimeService.save_account(
    channel='ibm_quantum',
    token='YOUR_TOKEN'
)

# After saving: use without args
service = QiskitRuntimeService()  # ✓ Uses saved credentials
```

This is the standard pattern after initial setup!
</details>

---

### Question 2: Backend Selection

**How do you get the least busy real quantum hardware?**

A) `service.backend('least_busy')`  
B) `service.least_busy(simulator=False, operational=True)`  
C) `service.get_best_backend()`  
D) `service.backends()[0]`

<details>
<summary>Answer</summary>

**B) `service.least_busy(simulator=False, operational=True)`**

```python
backend = service.least_busy(
    simulator=False,      # Exclude simulators
    operational=True      # Only working backends
)
```

Parameters:
- `simulator=False` → Real hardware only
- `operational=True` → Skip offline backends
- Returns backend with shortest queue
</details>

---

### Question 3: Backend Properties

**In Qiskit V2 API, how do you get supported operations?**

A) `backend.configuration().basis_gates`  
B) `backend.basis_gates`  
C) `backend.operation_names`  
D) `backend.get_operations()`

<details>
<summary>Answer</summary>

**C) `backend.operation_names`**

```python
# V2 API (Qiskit 1.0+)
ops = backend.operation_names  # ✓ Current
# Returns: ['ecr', 'id', 'rz', 'sx', 'x', 'measure', ...]

# V1 API (deprecated)
ops = backend.configuration().basis_gates  # ❌ Old
```

V2 API is simpler and more consistent!
</details>

---

### Question 4: Simulator Identification

**How can you identify if a backend is a simulator?**

A) Check if 'simulator' is in backend.name  
B) Use backend.is_simulator property  
C) Check if backend.num_qubits > 1000  
D) All of the above

<details>
<summary>Answer</summary>

**A) Check if 'simulator' is in backend.name**

```python
# Common pattern:
is_sim = 'simulator' in backend.name.lower()

# Examples:
'ibmq_qasm_simulator' → simulator ✓
'simulator_statevector' → simulator ✓  
'ibm_brisbane' → real hardware
```

Can also use `service.least_busy(simulator=True/False)` to filter!
</details>

---

## ✅ Key Takeaways

### Core Concepts

1. **QiskitRuntimeService** - Gateway to IBM Quantum
   - `save_account()` for one-time setup
   - `QiskitRuntimeService()` loads saved credentials
   - `channel='ibm_quantum'` for free tier

2. **Backend Selection**
   - `service.backend(name)` - Specific backend
   - `service.backends()` - List all
   - `service.least_busy()` - Auto-select

3. **Backend Properties** (V2 API)
   - `backend.name` - Backend name
   - `backend.num_qubits` - Qubit count
   - `backend.operation_names` - Supported operations
   - `backend.coupling_map` - Connectivity
   - `backend.target` - Detailed specs

4. **Simulator vs Hardware**
   - Simulators: 'simulator' in name, fast, ideal
   - Hardware: Real devices, noise, queues
   - Use `simulator=True/False` to filter

### Critical Exam Facts

- ✅ `QiskitRuntimeService()` uses saved credentials
- ✅ `save_account()` is one-time setup
- ✅ V2 API: `backend.operation_names` (not `basis_gates`)
- ✅ `least_busy()` finds shortest queue
- ✅ Simulators have 'simulator' in name
- ✅ `backend.run()` is DEPRECATED - use primitives!
- ✅ `channel='ibm_quantum'` for free tier

### Exam Patterns

**Q**: Initialize service?  
**A**: `QiskitRuntimeService()` after `save_account()`

**Q**: Get specific backend?  
**A**: `service.backend('ibm_brisbane')`

**Q**: Find best hardware?  
**A**: `service.least_busy(simulator=False, operational=True)`

**Q**: Get supported operations?  
**A**: `backend.operation_names` (V2 API)

### Mnemonic

🧠 **"Save Once, Load Always, Select Smart!"**

**Next**: Transpilation - converting circuits for hardware!